# Introduction

Using this Python notebook you will:

1.  Understand three Chicago datasets
2.  Load the three datasets into three tables in a SQLIte database
3.  Execute SQL queries to answer assignment questions

In [78]:
# Initialise

import csv, sqlite3

con = sqlite3.connect("project4.db")
cur = con.cursor()

In [79]:
%pip install pandas
%pip install ipython-sql prettytable

import prettytable
prettytable.DEFAULT = 'DEFAULT'


Note: you may need to restart the kernel to use updated packages.


In [80]:
%pip install ipython-sql
%load_ext sql

Note: you may need to restart the kernel to use updated packages.
The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [81]:
%sql sqlite:///project4.db

In [82]:
# Get Data into the SQL database from CSV

import pandas

df = pandas.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/FinalModule_Coursera_V5/data/ChicagoCensusData.csv?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkDB0201ENSkillsNetwork20127838-2021-01-01")
df.to_sql("CENSUS_DATA", con, if_exists='replace', index=False, method="multi")

df2 = pandas.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/FinalModule_Coursera_V5/data/ChicagoPublicSchools.csv?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkDB0201ENSkillsNetwork20127838-2021-01-01")
df2.to_sql("PUBLIC_SCHOOLS_DATA", con, if_exists='replace', index=False, method="multi")

df3 = pandas.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/FinalModule_Coursera_V5/data/ChicagoCrimeData.csv?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkDB0201ENSkillsNetwork20127838-2021-01-01")
df3.to_sql("CRIME_DATA", con, if_exists='replace', index=False, method="multi")

533

In [83]:
%sql SELECT name FROM sqlite_master WHERE type='table'

 * sqlite:///project4.db
Done.


name
CENSUS_DATA
PUBLIC_SCHOOLS_DATA
CRIME_DATA


In [84]:
%sql SELECT name FROM PRAGMA_TABLE_INFO('CRIME_DATA')

 * sqlite:///project4.db
Done.


name
ID
CASE_NUMBER
DATE
BLOCK
IUCR
PRIMARY_TYPE
DESCRIPTION
LOCATION_DESCRIPTION
ARREST
DOMESTIC


In [85]:
# Find the total number of crimes recorded in the CRIME table.

%sql SELECT COUNT(*) FROM CRIME_DATA

 * sqlite:///project4.db
Done.


COUNT(*)
533


In [86]:
%sql SELECT name FROM PRAGMA_TABLE_INFO('CENSUS_DATA')

 * sqlite:///project4.db
Done.


name
COMMUNITY_AREA_NUMBER
COMMUNITY_AREA_NAME
PERCENT_OF_HOUSING_CROWDED
PERCENT_HOUSEHOLDS_BELOW_POVERTY
PERCENT_AGED_16__UNEMPLOYED
PERCENT_AGED_25__WITHOUT_HIGH_SCHOOL_DIPLOMA
PERCENT_AGED_UNDER_18_OR_OVER_64
PER_CAPITA_INCOME
HARDSHIP_INDEX


In [87]:
# List community area names and numbers with per capita income less than 11000.

%sql SELECT COMMUNITY_AREA_NUMBER, COMMUNITY_AREA_NAME FROM CENSUS_DATA WHERE PER_CAPITA_INCOME < 11000

 * sqlite:///project4.db
Done.


COMMUNITY_AREA_NUMBER,COMMUNITY_AREA_NAME
26.0,West Garfield Park
30.0,South Lawndale
37.0,Fuller Park
54.0,Riverdale


In [88]:
# List all case numbers for crimes involving minors?(children are not considered minors for the purposes of crime analysis)

%sql SELECT CASE_NUMBER FROM CRIME_DATA WHERE PRIMARY_TYPE = 'OFFENSE INVOLVING CHILDREN'

 * sqlite:///project4.db
Done.


CASE_NUMBER
HN567387
HR391350
HM768251
HT394616


In [89]:
# List all kidnapping crimes involving a child?

%sql SELECT CASE_NUMBER, DESCRIPTION FROM CRIME_DATA WHERE PRIMARY_TYPE = 'KIDNAPPING' AND DESCRIPTION LIKE '%CHILD%'

 * sqlite:///project4.db
Done.


CASE_NUMBER,DESCRIPTION
HN144152,CHILD ABDUCTION/STRANGER


In [98]:
# List the kind of crimes that were recorded at schools. (No repetitions)

%sql SELECT DISTINCT(PRIMARY_TYPE) FROM CRIME_DATA WHERE LOCATION_DESCRIPTION LIKE '%SCHOOL%'

 * sqlite:///project4.db
Done.


PRIMARY_TYPE
BATTERY
CRIMINAL DAMAGE
NARCOTICS
ASSAULT
CRIMINAL TRESPASS
PUBLIC PEACE VIOLATION


In [91]:
# List the type of schools along with the average safety score for each type.

%sql SELECT `Elementary, Middle, or High School`, AVG(SAFETY_SCORE) FROM PUBLIC_SCHOOLS_DATA GROUP BY `Elementary, Middle, or High School`

 * sqlite:///project4.db
Done.


"Elementary, Middle, or High School",AVG(SAFETY_SCORE)
ES,49.52038369304557
HS,49.62352941176471
MS,48.0


In [92]:
# List 5 community areas with highest % of households below poverty line

%sql SELECT COMMUNITY_AREA_NAME, PERCENT_HOUSEHOLDS_BELOW_POVERTY FROM CENSUS_DATA ORDER BY PERCENT_HOUSEHOLDS_BELOW_POVERTY DESC LIMIT 5

 * sqlite:///project4.db
Done.


COMMUNITY_AREA_NAME,PERCENT_HOUSEHOLDS_BELOW_POVERTY
Riverdale,56.5
Fuller Park,51.2
Englewood,46.6
North Lawndale,43.1
East Garfield Park,42.4


In [93]:
# Which community area is most crime prone? Display the coumminty area number only.

%sql SELECT COMMUNITY_AREA_NUMBER, COUNT(*) AS 'TOTAL_CRIME' FROM CRIME_DATA WHERE COMMUNITY_AREA_NUMBER != 'None' GROUP BY COMMUNITY_AREA_NUMBER ORDER BY TOTAL_CRIME DESC LIMIT 1

 * sqlite:///project4.db
Done.


COMMUNITY_AREA_NUMBER,TOTAL_CRIME
25.0,43


In [101]:
# Use a sub-query to find the name of the community area with highest hardship index

%sql SELECT COMMUNITY_AREA_NAME FROM CENSUS_DATA WHERE HARDSHIP_INDEX IN (SELECT MAX(HARDSHIP_INDEX) FROM CENSUS_DATA)

 * sqlite:///project4.db
Done.


COMMUNITY_AREA_NAME
Riverdale


In [95]:
# Use a sub-query to determine the Community Area Name with most number of crimes?

In [96]:
%%sql

SELECT CD.COMMUNITY_AREA_NAME, COUNT(CRD.ID) AS 'TOTAL_CRIME' FROM CENSUS_DATA CD, CRIME_DATA CRD 
WHERE CD.community_area_number = CRD.community_area_number 
GROUP BY CD.COMMUNITY_AREA_NAME 
ORDER BY TOTAL_CRIME DESC 
LIMIT 1

 * sqlite:///project4.db
Done.


COMMUNITY_AREA_NAME,TOTAL_CRIME
Austin,43
